## Cardinality Clean up script

This script adds `owl:cardinality` information into the validation which will be ignored by any json schema validator (as this is the validation section is not the appropriate place for `owl:cardinality`. It is strictly being added to ensure ease of use by downstream scripts.

In [1]:
import json
import os
from biothings_schema import Schema

In [53]:
def fix_owl (bioschema_spec):
    context_spec = bioschema_spec['@context']
    context_spec['owl'] = 'http://www.w3.org/2002/07/owl/'
    return(context_spec)
    
def add_cardinality(propdict):
    propertylist = propdict.keys()
    for eachkey in propertylist:
        tmpdict = propdict[eachkey]
        dictstring = json.dumps(tmpdict)
        if "array" in dictstring:
            tmpdict['owl:cardinality'] = 'many'
        else:
            tmpdict['owl:cardinality'] = 'one'
        propdict[eachkey] = tmpdict
    return propdict  

def update_file(tmpdir,filelist):
    for eachfile in filelist:
        print("file opened: ",eachfile)
        with open(os.path.join(tmpdir,eachfile),'r', encoding='utf-8') as injson:
            bioschema = json.load(injson)
        newschema = {}
        newschema['@context'] = fix_owl(bioschema)
        graphlist = []
        exportflag=False
        for eachitem in bioschema['@graph']:
            if "$validation" in list(eachitem.keys()):
                propdict = eachitem['$validation']['properties']  
                propdict = add_cardinality(propdict)
                eachitem['$validation']['properties'] = propdict
                graphlist.append(eachitem)
                exportflag=True
            else:
                continue
        if exportflag == True:
            newschema['@graph'] = graphlist
            with open(os.path.join(tmpdir,eachfile),'w') as outjson:
                outjson.write(json.dumps(newschema, indent=4, sort_keys=False))
            
def update_specs(script_path):
    resulting_json = os.path.join(script_path,'results','resulting_json')
    specs = os.listdir(resulting_json)
    for specclass in specs:
        if '.txt' not in specclass:
            tmpdir = os.path.join(resulting_json,specclass,'jsonld')
            filelist = os.listdir(tmpdir)
            update_file(tmpdir,filelist)

def check_validation(script_path):
    resulting_json = os.path.join(script_path,'results','resulting_json')
    specs = os.listdir(resulting_json)
    for specclass in specs:
        print(specclass)
        tmpdir = os.path.join(resulting_json,specclass,'jsonld')
        filelist = os.listdir(tmpdir)
        for eachfile in filelist:
            print(eachfile)
            with open(os.path.join(tmpdir,eachfile),'r') as injson:
                bioschema = json.load(injson)
                sc = Schema(bioschema, base_schema=["schema.org","bioschemas"])
                sc.validation

In [51]:
## Main
script_path = ''
update_specs(script_path)

file opened:  Beacon_v0.2-DRAFT-2018_04_23-DEPRECATED.json
file opened:  BioChemEntity_v0.7-RELEASE.json
file opened:  BioChemEntity_v0.8-DRAFT.jsonld
file opened:  BioChemStructure_v0.1-DRAFT-2019_06_20.json
file opened:  BioSample_v0.1-RELEASE.json
file opened:  ChemicalSubstance_v0.1-DRAFT-2018_12_07.json
file opened:  ChemicalSubstance_v0.2-DRAFT-2019_06_11.json
file opened:  ChemicalSubstance_v0.3-DRAFT-2019_11_11.json
file opened:  ChemicalSubstance_v0.3-RELEASE-2019_09_02.json
file opened:  ChemicalSubstance_v0.4-RELEASE.json
file opened:  ComputationalTool_v0.5-DRAFT.json
file opened:  ComputationalTool_v1.0-RELEASE.json
file opened:  ComputationalWorkflow_v0.1-DRAFT-2019_02_08.json
file opened:  ComputationalWorkflow_v0.2-DRAFT-2019_11_29.json
file opened:  ComputationalWorkflow_v0.3-DRAFT-2020_03_03.json
file opened:  ComputationalWorkflow_v0.4-DRAFT-2020_05_11.json
file opened:  ComputationalWorkflow_v0.5-DRAFT-2020_07_21.json
file opened:  ComputationalWorkflow_v1.0-RELEASE

## Check validation for manual fixes

In [54]:
script_path = ''
check_validation(script_path)

Beacon
Beacon_v0.2-DRAFT-2018_04_23-DEPRECATED.json
BioChemEntity
BioChemEntity_v0.7-RELEASE.json
BioChemEntity_v0.8-DRAFT.jsonld
BioChemStructure
BioChemStructure_v0.1-DRAFT-2019_06_20.json
BioSample
BioSample_v0.1-RELEASE.json
ChemicalSubstance
ChemicalSubstance_v0.1-DRAFT-2018_12_07.json


SchemaValidationError: Class "http://discovery.biothings.io/view/bioschemas/ChemicalSubstance" has no path to the root "schema:Thing" class